In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName("weather").getOrCreate()

In [4]:
df = spark.read.csv("data/weather_dir/weather_data.csv", header=True, inferSchema=True)

#Нахождение среднего значения по столбцам

In [5]:
mean_temp = df.select(mean(df["temperature"])).collect()[0][0]
mean_prec = df.select(mean(df["precipitation"])).collect()[0][0]
mean_ws = df.select(mean(df["wind_speed"])).collect()[0][0]

Заполнение пустых значений на средние по каждому столбцу

In [6]:
df.na.fill({"temperature": mean_temp,
            "precipitation": mean_prec,
            "wind_speed": mean_ws})
df.show()

+----------+----------+-------------------+------------------+-------------------+
|station_id|      date|        temperature|     precipitation|         wind_speed|
+----------+----------+-------------------+------------------+-------------------+
| station_7|2022-06-28| -6.751842212861652| 23.67004407474563|  5.458999894629757|
| station_4|2020-04-07|  -9.57484426026233|2.9858244485444665|  6.828505392085966|
| station_8|2018-12-22|  19.34342035369938| 33.58211407730149|  8.975576079892296|
| station_5|2021-09-09| 30.880953114964086|29.110437988411558|  18.26465360842913|
| station_7|2023-02-07| 23.459048180599673|49.510444679381074|  3.787868357696922|
|station_10|2018-05-07|  27.52561802563259|11.957938889112151|0.10053105219606229|
| station_3|2019-08-27|  4.287905745111988|26.536867764341793|  1.201928207563403|
| station_7|2023-04-13| 19.244547722319886|32.547866755903634| 18.095326833602357|
| station_8|2020-10-10| 1.2541744266864718| 32.22955873711718| 12.950329296867263|
| st

Топ-5 самых жарких дней за всё время наблюдений

In [19]:
df.select(col("date"), col("temperature")).orderBy(col("temperature").desc()).limit(5).show()

+----------+------------------+
|      date|       temperature|
+----------+------------------+
|2021-08-20|39.982828249354846|
|2023-12-02| 39.96797489293784|
|2022-03-28|  39.8246894248997|
|2019-02-11| 39.76737697836647|
|2020-06-10| 39.69147838355929|
+----------+------------------+



 Метеостанция с наибольшим количеством осадков за последний год

In [17]:
last_year_df = df.where(year("date") == 2023)
station_precip = last_year_df.groupBy("station_id").sum("precipitation").orderBy(col("sum(precipitation)").desc()).limit(1)

station_precip.show()

+----------+------------------+
|station_id|sum(precipitation)|
+----------+------------------+
| station_5| 642.9302626767898|
+----------+------------------+



Средняя температура по месяцам

In [22]:
monthly_avg_temp = df.withColumn("month", month("date")).groupBy("month").avg("temperature").orderBy("month")
monthly_avg_temp.show()

+-----+------------------+
|month|  avg(temperature)|
+-----+------------------+
|    1|11.356518462550754|
|    2| 9.067229891101926|
|    3| 7.244080205633994|
|    4|12.024529009744693|
|    5| 9.902883346912718|
|    6|13.421092297254138|
|    7|6.1857183016954576|
|    8|  10.9678002814186|
|    9| 9.596744236573942|
|   10|  9.09884344821895|
|   11| 7.265889994697494|
|   12|11.218592100674337|
+-----+------------------+

